In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

import cv2
import pandas as pd
from contextlib import contextmanager
import copy
from dataclasses import dataclass
import gc 
import os
from pathlib import Path
import random
import sys
import time

In [2]:
@dataclass
class Config():
    ### Config for PyTorch
    seed: int = 20230316
    debug: bool = False
    logger_fname: str = "412_mnist.log"
    model_dir: str = "/content/drive/MyDrive/Computing Science/CMPUT 412/Assign 5/photo_weights/"
    model_file_prefix: str = "cnn_model"
    root: str = ".data"
    df_path: str = "/content/drive/MyDrive/Computing Science/CMPUT 412/Assign 5/mod_data.csv"

    img_dir: str = "/content/drive/MyDrive/Computing Science/CMPUT 412/Assign 5/samples/"
    batch_size: int = 8
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    lr: float = 1e-5
    dropout: float = 0.4
    epochs: int = 15

    n_folds: int = 5

In [3]:
### Util functions ###
def seed_everything(seed=Config.seed):
    # Initialize seeds for every dependencies
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    # Only when we use deep learning
    if "torch" in sys.modules:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

In [4]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.RandomAffine(10, scale=(0.95, 1.05), shear=15),
                                       transforms.Resize((42, 42)),
                                       transforms.Normalize(mean = [0.5], std = [0.1])])

test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((42, 42)),
                                      transforms.Normalize(mean = [0.5], std = [0.1])])

In [5]:
class Data(data.Dataset):
    def __init__(self, df, is_train, trans):
        self.df = df
        self.is_train = is_train
        self.transform = trans
        
    def __len__(self):
        return self.df.shape[0]

    def read_img(self, img_path):
        img = cv2.imread(img_path)
        
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        im = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        lower_blue = np.array([80,120,0])
        upper_blue = np.array([110,255,255])
        msk = cv2.inRange(im, lower_blue, upper_blue)

        contours, _ = cv2.findContours(msk,
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_NONE)

        max_area = 20
        max_idx = -1
        for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if area > max_area:
                max_idx = i
                max_area = area

        b = np.zeros_like(msk)
        cv2.fillPoly(b, [contours[max_idx]], 255)
        sel = b != 255

        msk[sel] = 255
        msk = 255 - msk
        
        msk = self.transform(msk)
                
        return msk
        
    def __getitem__(self, idx):
        data = self.df.loc[idx, :]
        img_path = Config.img_dir + str(int(data["idx"])) + ".png"
        img = self.read_img(img_path)

        if self.is_train:
            return (img, data["label"])
        else:
            return img

In [6]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Two layers of CNN and single dense layer
        self.conv1 = nn.Conv2d(1, 32, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.bnorm1 = nn.BatchNorm2d(32)
        self.bnorm2 = nn.BatchNorm2d(64)
        self.mx_pool = nn.MaxPool2d(3)
        self.dp1 = nn.Dropout(0.3)
        self.dp2 = nn.Dropout(0.6)
        self.fc1 = nn.Linear(12544, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size = x.shape[0]

        x = F.relu(self.conv1(x))
        x = self.bnorm1(x)
        x = F.relu(self.conv2(x))
        x = self.bnorm2(x)
        x = self.mx_pool(x)

        x = self.dp1(x)
        x = x.view(batch_size, 1, -1)
        x = F.relu(self.fc1(x))
        # x = self.bnorm3(x)

        x = self.dp2(x)
        x = self.fc2(x)

        return x

class DenseModel(nn.Module):
    def __init__(self, in_dim=784):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(784, 350)
        self.fc2 = nn.Linear(350, 100)
        self.fc3 = nn.Linear(100, 10)
        self.dp = nn.Dropout(0.4)
        self.bnorm = nn.BatchNorm1d(1)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, 1, -1)

        x = F.relu(self.fc1(x))
        x = self.dp(x)
        x = self.bnorm(x)

        x = F.relu(self.fc2(x))
        x = self.dp(x)
        x = self.bnorm(x)
        
        x = self.fc3(x)
        
        return x

In [7]:
def to_dl(dataset, shuffle=False):
    return data.DataLoader(dataset,
                           batch_size=Config.batch_size,
                           shuffle=shuffle,
                           num_workers=4,
                           pin_memory=True,)
    
def get_acc(pred, y):
    top_pred = pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [8]:
def train_fold(train_dl, eval_dl, fold):
    # Prepare model
    model = CNNModel().to(Config.device)
    loss_fn = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=Config.lr)

    mx_acc, mn_loss = -1, 1e9
    best_param = model.state_dict()
    for ep in range(Config.epochs):
        # Training loop
        model.train()
        train_ep_loss = 0
        train_ep_acc = 0
        for img, label in tqdm(train_dl):
            img, label = img.to(Config.device), label.to(Config.device)
            pred = model(img.float()).squeeze(1)
            loss = loss_fn(pred, label.long())
            opt.zero_grad()
            loss.backward()
            opt.step()
            train_ep_loss += loss.item()
            train_ep_acc += get_acc(pred, label).item()
        
        train_ep_loss /= len(train_dl)
        train_ep_acc /= len(train_dl)
        
        # Evaluate with hold-out fold
        model.eval()
        eval_ep_loss = 0
        eval_ep_acc = 0
        with torch.no_grad():
            for img, label in tqdm(eval_dl):
                img, label = img.to(Config.device), label.to(Config.device) 
                pred = model(img.float()).squeeze(1)
                eval_ep_loss += loss_fn(pred, label.long()).item()
                eval_ep_acc += get_acc(pred, label).item()

        eval_ep_loss /= len(eval_dl)
        eval_ep_acc /= len(eval_dl)

        print(f"Epoch {ep+1}")
        print(f"\tTrain loss: {train_ep_loss} | Train Acc: {train_ep_acc}")
        print(f"\tEvaluation loss: {eval_ep_loss} | Evaluation Acc: {eval_ep_acc}")
        
        if eval_ep_loss < mn_loss:
            # Update the best weight if better loss is met
            mn_loss = eval_ep_loss
            torch.save(model.state_dict(), f"{Config.model_dir}/{Config.model_file_prefix}_{fold}.pth")
        
    del model, opt, loss_fn
    gc.collect()

def run_train(df):
    skf = StratifiedKFold(n_splits=Config.n_folds)
    for i, (_, test_idx) in enumerate(skf.split(df, df["label"])):
        df.loc[test_idx, "fold"] = i 
    
    for fold in range(Config.n_folds):
        print(f"---- Fold #{fold+1} ----")
        tr_data = Data(df[df["fold"] != fold].reset_index(drop=True), True, train_transforms)
        eval_data = Data(df[df["fold"] == fold].reset_index(drop=True), True, test_transforms)
        
        train_dl = to_dl(tr_data, shuffle=True)
        eval_dl = to_dl(eval_data)

        # Prepare model
        train_fold(train_dl, eval_dl, fold+1)

        del train_dl, eval_dl
        gc.collect()

def run_test(test_data, model_paths):
    te_data = data.TensorDataset(test_data.data, test_data.targets)
    test_dl = to_dl(te_data)
    model = CNNModel().to(Config.device)
    loss_fn = nn.CrossEntropyLoss()

    overall_loss, overall_acc = 0, 0
    for path in model_paths:
        model.load_state_dict(torch.load(path))
        model.eval()
        
        loss, acc = 0, 0
        with torch.no_grad():
            for img, label in tqdm(test_dl):
                img, label = img.to(Config.device), label.to(Config.device)
                pred = model(img.float()).squeeze(1)
                loss += loss_fn(pred, label)
                acc += get_acc(pred, label).item()
            
            loss /= len(test_dl)
            acc /= len(test_dl)
        
        print(f"Test loss for model {path}: {loss}")
        print(f"Test accuracy for model {path}: {acc}")
        overall_loss += loss / len(model_paths)
        overall_acc += acc / len(model_paths)
    
    print(f"Test loss: {overall_loss}")
    print(f"Test accuracy: {overall_acc}")
    
    del model, loss_fn, test_dl
    gc.collect()

In [9]:
# Run training and test
seed_everything()

df = pd.read_csv(Config.df_path)
run_train(df)

---- Fold #1 ----


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1
	Train loss: 2.2325278329849243 | Train Acc: 0.19375
	Evaluation loss: 1.749674391746521 | Evaluation Acc: 0.525


...

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 14
	Train loss: 0.6583265049010515 | Train Acc: 0.79625
	Evaluation loss: 0.43166655898094175 | Evaluation Acc: 0.895


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 15
	Train loss: 0.5849703305959701 | Train Acc: 0.83875
	Evaluation loss: 0.41297211855649946 | Evaluation Acc: 0.9


In [10]:
# weights_path = list(Path(Config.model_dir).glob(f"{Config.model_file_prefix}*.pth"))
# run_test(test_data, weights_path)